# Comparison of Soft Masks (b/w Mask RCNN and Mask Prop)

In [ ]:
import sys
sys.path.append('../')

import matplotlib.pyplot as plt
import numpy as np
import os
from keras.backend import tf

# static GPU memory allocation for tensorflow (reserve some GPU for PyTorch optical flow)
gpu_options = tf.GPUOptions(per_process_gpu_memory_fraction=0.75)
sess = tf.Session(config=tf.ConfigProto(gpu_options=gpu_options))

from image_seg.model import MaskRCNN
from mask_prop.mask_propagation import MaskPropagation

# Root directory of the project
ROOT_DIR = os.path.abspath("../")
print('Project Directory: {}'.format(ROOT_DIR))

# Root directory of the dataset
DATA_DIR = os.path.join(ROOT_DIR, "dataset/wad")
print('Data Directory: {}'.format(DATA_DIR))

# Directory to save logs and trained model
LOGS_DIR = os.path.join(ROOT_DIR, "logs")
print('Logs and Model Directory: {}'.format(LOGS_DIR))

In [ ]:
from train.davis_data import MaskPropDavisDataset
from skimage.io import imread

dataset = MaskPropDavisDataset("../../data/DAVIS", "480p", val_videos=[
    "car-shadow", "breakdance", "camel", "scooter-black", "libby", "drift-straight"
])

img_prev, img_curr, mask_prev, mask_curr =\
    list(map(imread, dataset.get_random_pair(val=False)))


## Mask RCNN Inference

In [ ]:
from image_seg import visualize
from train.wad_data import WadConfig, index_to_class_names


# WAD Configuration
class WADInferenceConfig(WadConfig):
    BATCH_SIZE = 1
    IMAGES_PER_GPU = 1
    DETECTION_MIN_CONFIDENCE = 0
    
    
wad_config = WADInferenceConfig()

MODEL_WEIGHTS_PATH = os.path.join(LOGS_DIR, 'wad20180621T1404/mask_rcnn_wad_0001.h5')

# Create model in inference mode
mrcnn = MaskRCNN(mode="inference", config=wad_config, model_dir=LOGS_DIR)

mrcnn.load_weights(MODEL_WEIGHTS_PATH, by_name=True)

results = mrcnn.detect([img_curr], verbose=1)[0]

visualize.display_instances(img_curr, results['rois'], results['masks'], results['class_ids'], 
                            index_to_class_names, results['scores'])

## Mask Propagation Inference

In [ ]:
from mask_prop.mask_propagation import pad_image
from opt_flow.pwc_net_wrapper import PWCNetWrapper

optical_flow = PWCNetWrapper("../opt_flow/pwc_net.pth.tar")
mp = MaskPropagation('../mask_prop/davis_unet_weights.h5')

# get the flow field
flow_field = optical_flow.infer_flow_field(img_prev, img_curr)
flow_field_x, flow_field_y = flow_field[:, :, 0], flow_field[:, :, 1]
flow_field[:, :, 0] = (flow_field_x - flow_field_x.mean()) / flow_field_x.std()
flow_field[:, :, 1] = (flow_field_y - flow_field_y.mean()) / flow_field_y.std()

inputs = np.stack([mask_prev, flow_field[:, :, 0], flow_field[:, :, 1]], axis=2)

# propagate the mask using flow field and previous mask
propagated_mask = mp.predict(inputs)

plt.imshow(propagated_mask)